In [1]:
import syft as sy
import numpy as np
import pandas as pd
from syft.core.adp.data_subject_list import DataSubjectList
from time import time, sleep

Loaded constant2epsilon cache of size: (300000,)


In [2]:
t0 = time()
df = pd.read_parquet("1M_rows_dataset_sample.parquet")
tf = time() - t0

print(f"Time taken to read file with {df.shape[0]/1e6} million rows : {tf} seconds")

Time taken to read file with 1.0 million rows : 0.2118072509765625 seconds


In [3]:
df.head()

,tweet_id,impressions,tweet_date_time,date,time,user_id,url,publication_title,ad_fontes_bias,ad_fontes_reliability,domain
0,1,9029,2021-03-21 06:29:10,2021-03-21,06:29:10,92256,https://www.aljazeera.com/news/2019/04/trump-v...,Al Jazeera,-0.75,48.0,.aljazeera.com
1,1,9029,2021-03-21 06:29:10,2021-03-21,06:29:10,418264,https://www.aljazeera.com/news/2019/04/trump-v...,Al Jazeera,-0.75,48.0,.aljazeera.com
2,1,9029,2021-03-21 06:29:10,2021-03-21,06:29:10,195053,https://www.aljazeera.com/news/2019/04/trump-v...,Al Jazeera,-0.75,48.0,.aljazeera.com
3,1,9029,2021-03-21 06:29:10,2021-03-21,06:29:10,590420,https://www.aljazeera.com/news/2019/04/trump-v...,Al Jazeera,-0.75,48.0,.aljazeera.com
4,1,9029,2021-03-21 06:29:10,2021-03-21,06:29:10,602475,https://www.aljazeera.com/news/2019/04/trump-v...,Al Jazeera,-0.75,48.0,.aljazeera.com


In [4]:
t0 = time()
impressions = df['impressions']
data_subjects = DataSubjectList.from_series(df['user_id'])
tf = time() - t0

print(f"Time taken to create inputs for Syft Tensor: {tf} seconds")

Time taken to create inputs for Syft Tensor: 0.18091416358947754 seconds


In [5]:
t0 = time()
tweets_data = sy.Tensor(impressions).private(min_val=500, max_val=25000, entities = data_subjects, ndept=True)  # Run this for 1 million rows
tf = time() - t0

print(f"Time make Private Syft Tensor: {tf} seconds")

Time make Private Syft Tensor: 0.001383066177368164 seconds


In [6]:
domain_node = sy.login(email="info@openmined.org",password="changethis",port=8081)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into peaceful_li... done!


In [7]:
%%time
name = f"Tweets data- {time()}"

domain_node.load_dataset(
    assets={name: tweets_data},
    name=name,
    description=" Tweets- 1B rows",
    use_blob_storage=True
)

Loading dataset... uploading...🚀                                                                                                                                             

kj/filesystem-disk-unix.c++:1690: warning: PWD environment variable doesn't match current directory; pwd = /home/ruchi/PySyft
Uploading `Tweets data- 1649425899.1405215`: 100%|████████████████████| 1/1 [00:00<00:00, 32.79it/s]

Dataset is uploaded successfully !!! 🎉

Run `<your client variable>.datasets` to see your new dataset loaded into your machine!
CPU times: user 206 ms, sys: 20.4 ms, total: 226 ms
Wall time: 361 ms


In [8]:
domain_node.datasets

Idx,Name,Description,Assets,Id
[0],Tweets data- 1649425561.4517217,Tweets- 1B rows,"[""Tweets data- 1649425561.4517217""] ->",dcdbab58-f720-432d-ac5c-32b2be1cb5dd
[1],Tweets data- 1649425899.1405215,Tweets- 1B rows,"[""Tweets data- 1649425899.1405215""] ->",b2a553db-dd1b-4ec8-a003-f0aa2ed72e45


In [14]:
data = domain_node.datasets[-1][name]

In [15]:
%%time
sum_result = data.sum()
sum_result.block

CPU times: user 22.9 ms, sys: 721 µs, total: 23.6 ms
Wall time: 324 ms


<TensorPointer -> adp:4a0558d016024720b183f42b4a140bea>

In [29]:
sum_result.exists

True

In [36]:
published_result = sum_result.publish(sigma=1e3)

Sigma, PB_spend
- 100, 859.4086891067999
- 1e4, 7.766216625311721
- 1e5, 7.766216625311721
- 2e5, 7.766216625311721
- 5e5, 7.766216625311721
- 1e7, 7.766216625311721
- 1e9, 7.766216625311721

Is PB not being calculated correctly?

In [37]:
published_result.exists

True

In [38]:
result_of_our_hard_labour = published_result.get(delete_obj=False)

In [39]:
result_of_our_hard_labour

array(-449.15066734)

In [40]:
impressions.sum()

11941995258

In [41]:
def percentage_error(true_value, noisy_value):
    return (true_value - noisy_value)/true_value * 100

In [42]:
percentage_error(true_value=impressions.sum(), noisy_value=result_of_our_hard_labour)

100.00000376110239

In [44]:
domain_node.privacy_budget

5.55